In [ ]:
!pip install konlpy
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.6 MB/s eta 0:00:00


In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

# document 열과 label 열의 중복을 제외한 값의 개수
train_data['document'].nunique(), train_data['label'].nunique()
# document 열의 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
train_data.loc[train_data.document.isnull()]
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거

# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1

tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

max_len = 30
below_threshold_len(max_len, X_train)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

<ipython-input-3-74add8a41e73>:26: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-3-74add8a41e73>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
<ipython-input-3-74add8a41e73>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
<ipython-input-3-74add8a41e73>:32: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
100%|██████████| 48852/48852 [05:07<00:00, 159.12it/s]
/usr/local/lib/python3.10/dist-packages/numpy/lib/functio

전체 샘플 중 길이가 30 이하인 샘플의 비율: 94.31944999380003
Epoch 1/15
1815/1815 [==============================] - ETA: 0s - loss: 0.4110 - acc: 0.8088
Epoch 1: val_acc improved from -inf to 0.83639, saving model to best_model.h5
1815/1815 [==============================] - 202s 108ms/step - loss: 0.4110 - acc: 0.8088 - val_loss: 0.3709 - val_acc: 0.8364
Epoch 2/15
1815/1815 [==============================] - ETA: 0s - loss: 0.3458 - acc: 0.8491
Epoch 2: val_acc improved from 0.83639 to 0.84345, saving model to best_model.h5
1815/1815 [==============================] - 180s 99ms/step - loss: 0.3458 - acc: 0.8491 - val_loss: 0.3579 - val_acc: 0.8435
Epoch 3/15
1815/1815 [==============================] - ETA: 0s - loss: 0.3190 - acc: 0.8635
Epoch 3: val_acc improved from 0.84345 to 0.85017, saving model to best_model.h5
1815/1815 [==============================] - 175s 96ms/step - loss: 0.3190 - acc: 0.8635 - val_loss: 0.3415 - val_acc: 0.8502
Epoch 4/15
1815/1815 [==============================] - E

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

1527/1527 [==============================] - 36s 23ms/step - loss: 0.3476 - acc: 0.8525

 테스트 정확도: 0.8525


In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.6):
    print("{:.2f}% 확률로 긍정 문장입니다.\n".format(score * 100))
  elif(score < 0.4):
    print("{:.2f}% 확률로 부정 문장입니다.\n".format((1 - score) * 100))
  elif(0.4<=score<=0.6):
    print("중립 문장입니다.")

while(1):
  answer= input("문장을 입력해주세요.")
  reply = sentiment_predict(answer)

1/1 [==============================] - 0s 453ms/step
중립 문장입니다.
1/1 [==============================] - 0s 25ms/step
68.96% 확률로 긍정 문장입니다.

1/1 [==============================] - 0s 25ms/step
90.64% 확률로 긍정 문장입니다.

1/1 [==============================] - 0s 30ms/step
79.69% 확률로 긍정 문장입니다.

1/1 [==============================] - 0s 27ms/step
70.83% 확률로 긍정 문장입니다.

1/1 [==============================] - 0s 43ms/step
73.65% 확률로 부정 문장입니다.

1/1 [==============================] - 0s 26ms/step
74.11% 확률로 부정 문장입니다.

1/1 [==============================] - 0s 24ms/step
중립 문장입니다.
1/1 [==============================] - 0s 30ms/step
75.02% 확률로 부정 문장입니다.

1/1 [==============================] - 0s 26ms/step
중립 문장입니다.
1/1 [==============================] - 0s 35ms/step
85.04% 확률로 부정 문장입니다.

1/1 [==============================] - 0s 29ms/step
중립 문장입니다.
1/1 [==============================] - 0s 27ms/step
98.90% 확률로 부정 문장입니다.

1/1 [==============================] - 0s 26ms/step
96.16% 확률로 부정 문장입니다.

1/1 [======